In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


/usr/local/Cellar/python3/3.6.1/Frameworks/Python.framework/Versions/3.6/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
# Basic tf setting
tf.set_random_seed(2016)
sess = tf.Session()

# Get data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [8]:
a = mnist.train.images

In [11]:
import numpy as np

In [12]:
a.shape

(55000, 784)

In [13]:
b = mnist.test.images

In [14]:
b.shape

(10000, 784)

In [15]:
train_x = mnist.train.images
train_y = mnist.train.labels

In [16]:
test_x = mnist.test.images
test_y = mnist.test.labels

In [34]:
C = 1

class ELM(object):
    def __init__(self,sess,batch_size,input_len,hidden_len,output_len):
        
        self.sess = sess 
        self.batch_size = batch_size
        self.input_len = input_len
        self.hidden_len = hidden_len
        self.output_len = output_len
        
        #Training variables
        self.x = tf.placeholder(tf.float32, [self.batch_size, self.input_len])
        self.t = tf.placeholder(tf.float32, [self.batch_size, self.output_len])
        
        #Testing variables 
        self.x_test = tf.placeholder(tf.float32,[None,self.input_len])
        self.y_test = tf.placeholder(tf.float32,[None,self.output_len])
        
        #Weight , bias and beta Variable 
        self.W = tf.Variable(tf.random_normal([self.input_len, self.hidden_len]),trainable=False, dtype=tf.float32)
        self.b = tf.Variable(tf.random_normal([self.hidden_len]),trainable=False, dtype=tf.float32)
        self.beta = tf.Variable(tf.zeros([self.hidden_len, self.output_len]),trainable=False, dtype=tf.float32)
        self.var_list = [self.W, self.b, self.beta]
        
        #Training hidden outputs
        self.h = tf.matmul(self.x, self.W) + self.b
        self.h_t = tf.transpose(self.h)
        
        #Testing hidden outputs
        self.h_test = tf.matmul(self.x_test, self.W) + self.b 
        self.h_test_t = tf.transpose(self.h_test)
        
        #Finding beta
        if self.input_len < self.hidden_len:  # D < L
            identity = tf.constant(np.identity(self.hidden_len), dtype=tf.float32)
            self.beta_a = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(self.h_t, self.h) + identity/C),self.h_t), self.t)
            # beta_a = (H_T*H + I/C)^(-1)*H_T*T
        else:
            identity = tf.constant(np.identity(self.batch_size), dtype=tf.float32)
            self.beta_a = tf.matmul(tf.matmul(self.h_t,tf.matrix_inverse(tf.matmul(self.h, self.h_t)+identity/C)), self.t)
            # beta_a = H_T*(H*H_T + I/C)^(-1)*T
        
        
        self.assign_beta = self.beta.assign(self.beta_a)
        self.out = tf.matmul(self.h, self.beta)
        self.out_test = tf.matmul(self.h_test, self.beta)
        
        self.cost = tf.reduce_mean(tf.losses.mean_squared_error(labels = self.t,predictions = self.out))

        self._init = False
        self._feed = False

        # for the mnist test
        self.correct_prediction = tf.equal(tf.argmax(self.out_test,1), tf.argmax(self.y_test,1))
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

    def feed(self, x, t):
        
        if not self._init : self.init()
        self.sess.run(self.assign_beta, {self.x:x, self.t:t})
        self._feed = True

    def init(self):
        
        self.sess.run(tf.initialize_variables(self.var_list))
        self._init = True

    def test(self, x, t=None):
        
        if not self._feed : exit("Not feed-forward trained")
        
        if t is not None :
            print("Accuracy: {:.9f}".format(self.sess.run(self.accuracy, {self.x_test:x, self.y_test:t})))
        else :
            return self.sess.run(self.out_test, {self.x_test:x})
        

In [35]:
batch_size = 5000
hidden_num = 125
print("batch_size : {}".format(batch_size))
print("hidden_num : {}".format(hidden_num))
elm = ELM(sess, batch_size, 784, hidden_num, 10)

# one-step feed-forward training
train_x, train_y = mnist.train.next_batch(batch_size)
elm.feed(train_x, train_y)

# testing
elm.test(mnist.test.images, mnist.test.labels)

batch_size : 5000
hidden_num : [125]


ValueError: setting an array element with a sequence.

In [33]:
a = [1]
a[-1]

1